In [1]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [2]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [3]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [4]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (datetime.datetime.today(), json.dumps(neo)))
    conn.commit()

load_data()

Exploration of SQL transformation

In [5]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [6]:
df = pd.read_sql_query('''SELECT * FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3341510,3341510,(2006 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.200,False,"[{'close_approach_date': '2024-02-18', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3341510?ap...,0.009651,0.021579,9.650615,21.579430,0.005997,0.013409,31.662123,70.798659
1,3601342,3601342,(2012 EZ11),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.490,False,"[{'close_approach_date': '2024-02-18', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3601342?ap...,0.133830,0.299254,133.830462,299.254010,0.083158,0.185948,439.076332,981.804526
2,3743123,3743123,(2016 CA138),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.290,False,"[{'close_approach_date': '2024-02-18', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3743123?ap...,0.058419,0.130629,58.419115,130.629113,0.036300,0.081169,191.663771,428.573220
3,3840791,3840791,(2019 GM1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.250,False,"[{'close_approach_date': '2024-02-18', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3840791?ap...,0.009431,0.021088,9.430940,21.088223,0.005860,0.013104,30.941405,69.187084
4,54226416,54226416,(2021 XT1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,28.600,False,"[{'close_approach_date': '2024-02-18', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54226416?a...,0.005065,0.011325,5.064715,11.325046,0.003147,0.007037,16.616518,37.155664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14321,54421913,54421913,(2024 AN6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,19.778,False,"[{'close_approach_date': '2024-02-12', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54421913?a...,0.294412,0.658324,294.411640,658.324441,0.182939,0.409064,965.917485,2159.857158
14322,54423325,54423325,(2024 BH5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.841,False,"[{'close_approach_date': '2024-02-12', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54423325?a...,0.045327,0.101354,45.326791,101.353787,0.028165,0.062978,148.709950,332.525557
14323,54425949,54425949,(2024 CY1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,29.211,False,"[{'close_approach_date': '2024-02-12', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54425949?a...,0.003823,0.008548,3.822566,8.547517,0.002375,0.005311,12.541227,28.043036
14324,54426469,54426469,(2024 CC4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.493,False,"[{'close_approach_date': '2024-02-12', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54426469?a...,0.008432,0.018856,8.432473,18.855584,0.005240,0.011716,27.665596,61.862153


Extracting nested close approach data

In [7]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2024-02-18',
  'close_approach_date_full': '2024-Feb-18 01:36',
  'epoch_date_close_approach': 1708220160000,
  'relative_velocity': {'kilometers_per_second': '6.5483451916',
   'kilometers_per_hour': '23574.0426897212',
   'miles_per_hour': '14647.9933590659'},
  'miss_distance': {'astronomical': '0.1585031059',
   'lunar': '61.6577081951',
   'kilometers': '23711727.031024433',
   'miles': '14733783.9624378554'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [8]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_19280\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3341510,3341510,(2006 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.200,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3341510?ap...,0.009651,0.021579,9.650615,21.579430,0.005997,0.013409,31.662123,70.798659
1,3601342,3601342,(2012 EZ11),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.490,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3601342?ap...,0.133830,0.299254,133.830462,299.254010,0.083158,0.185948,439.076332,981.804526
2,3743123,3743123,(2016 CA138),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.290,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3743123?ap...,0.058419,0.130629,58.419115,130.629113,0.036300,0.081169,191.663771,428.573220
3,3840791,3840791,(2019 GM1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.250,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3840791?ap...,0.009431,0.021088,9.430940,21.088223,0.005860,0.013104,30.941405,69.187084
4,54226416,54226416,(2021 XT1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,28.600,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54226416?a...,0.005065,0.011325,5.064715,11.325046,0.003147,0.007037,16.616518,37.155664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14321,54421913,54421913,(2024 AN6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,19.778,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54421913?a...,0.294412,0.658324,294.411640,658.324441,0.182939,0.409064,965.917485,2159.857158
14322,54423325,54423325,(2024 BH5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.841,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54423325?a...,0.045327,0.101354,45.326791,101.353787,0.028165,0.062978,148.709950,332.525557
14323,54425949,54425949,(2024 CY1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,29.211,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54425949?a...,0.003823,0.008548,3.822566,8.547517,0.002375,0.005311,12.541227,28.043036
14324,54426469,54426469,(2024 CC4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.493,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54426469?a...,0.008432,0.018856,8.432473,18.855584,0.005240,0.011716,27.665596,61.862153


In [9]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2024-02-18,2024-Feb-18 01:36,1708220160000,Earth,6.5483451916,23574.0426897212,14647.9933590659,0.1585031059,61.6577081951,23711727.031024433,14733783.9624378554
1,2024-02-18,2024-Feb-18 11:23,1708255380000,Earth,10.3688473103,37327.8503169828,23194.0745483497,0.2752193896,107.0603425544,41172234.466860152,25583240.1870960176
2,2024-02-18,2024-Feb-18 16:44,1708274640000,Earth,14.3452175429,51642.7831546067,32088.8171218166,0.0644683338,25.0781818482,9644325.418929006,5992705.9298560428
3,2024-02-18,2024-Feb-18 22:57,1708297020000,Earth,3.4570904169,12445.5255007355,7733.1655534987,0.1311391528,51.0131304392,19618137.932484536,12190145.6466811568
4,2024-02-18,2024-Feb-18 11:40,1708256400000,Earth,20.0489643477,72176.2716517384,44847.5283493815,0.4810100214,187.1128983246,71958074.650094418,44712674.2333142484
...,...,...,...,...,...,...,...,...,...,...,...
14321,2024-02-12,2024-Feb-12 14:24,1707747840000,Earth,30.8939918704,111218.370733327,69106.7704148275,0.4812669927,187.2128601603,71996517.009225549,44736561.2076246162
14322,2024-02-12,2024-Feb-12 14:56,1707749760000,Earth,11.817004908,42541.2176686402,26433.4582786059,0.0936693998,36.4373965222,14012742.694258426,8707114.5559452388
14323,2024-02-12,2024-Feb-12 07:32,1707723120000,Earth,14.5258773843,52293.1585835866,32492.9351210069,0.0008119411,0.3158450879,121464.659125457,75474.6393750266
14324,2024-02-12,2024-Feb-12 04:17,1707711420000,Earth,7.7637281294,27949.4212658338,17366.683452651,0.01176307,4.57583423,1759730.2166609,1093445.65288442


In [10]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,date_added,data
0,3341510,3341510,(2006 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.200,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3341510?ap...,0.009651,...,Earth,6.5483451916,23574.0426897212,14647.9933590659,0.1585031059,61.6577081951,23711727.031024433,14733783.9624378554,2024-02-19 17:05:26.730419,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3601342,3601342,(2012 EZ11),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.490,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3601342?ap...,0.133830,...,Earth,10.3688473103,37327.8503169828,23194.0745483497,0.2752193896,107.0603425544,41172234.466860152,25583240.1870960176,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3743123,3743123,(2016 CA138),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.290,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3743123?ap...,0.058419,...,Earth,14.3452175429,51642.7831546067,32088.8171218166,0.0644683338,25.0781818482,9644325.418929006,5992705.9298560428,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3840791,3840791,(2019 GM1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.250,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3840791?ap...,0.009431,...,Earth,3.4570904169,12445.5255007355,7733.1655534987,0.1311391528,51.0131304392,19618137.932484536,12190145.6466811568,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,54226416,54226416,(2021 XT1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,28.600,False,"{'close_approach_date': '2024-02-18', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54226416?a...,0.005065,...,Earth,20.0489643477,72176.2716517384,44847.5283493815,0.4810100214,187.1128983246,71958074.650094418,44712674.2333142484,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14321,54421913,54421913,(2024 AN6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,19.778,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54421913?a...,0.294412,...,Earth,30.8939918704,111218.370733327,69106.7704148275,0.4812669927,187.2128601603,71996517.009225549,44736561.2076246162,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14322,54423325,54423325,(2024 BH5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.841,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54423325?a...,0.045327,...,Earth,11.817004908,42541.2176686402,26433.4582786059,0.0936693998,36.4373965222,14012742.694258426,8707114.5559452388,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14323,54425949,54425949,(2024 CY1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,29.211,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54425949?a...,0.003823,...,Earth,14.5258773843,52293.1585835866,32492.9351210069,0.0008119411,0.3158450879,121464.659125457,75474.6393750266,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14324,54426469,54426469,(2024 CC4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.493,False,"{'close_approach_date': '2024-02-12', 'close_a...",False,http://api.nasa.gov/neo/rest/v1

Cleaning up df

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14326 entries, 0 to 14325
Data columns (total 30 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    14326 non-null  object 
 1   neo_reference_id                                      14326 non-null  object 
 2   name                                                  14326 non-null  object 
 3   nasa_jpl_url                                          14326 non-null  object 
 4   absolute_magnitude_h                                  14324 non-null  float64
 5   is_potentially_hazardous_asteroid                     14326 non-null  bool   
 6   close_approach_data                                   14326 non-null  object 
 7   is_sentry_object                                      14326 non-null  bool   
 8   links.self                                            14

In [12]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,date_added,data
0,3341510,(2006 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.200,False,False,9.650615,21.579430,2024-Feb-18 01:36,Earth,6.5483451916,23574.0426897212,0.1585031059,61.6577081951,23711727.031024433,2024-02-19 17:05:26.730419,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3601342,(2012 EZ11),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.490,False,False,133.830462,299.254010,2024-Feb-18 11:23,Earth,10.3688473103,37327.8503169828,0.2752193896,107.0603425544,41172234.466860152,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3743123,(2016 CA138),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.290,False,False,58.419115,130.629113,2024-Feb-18 16:44,Earth,14.3452175429,51642.7831546067,0.0644683338,25.0781818482,9644325.418929006,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3840791,(2019 GM1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.250,False,False,9.430940,21.088223,2024-Feb-18 22:57,Earth,3.4570904169,12445.5255007355,0.1311391528,51.0131304392,19618137.932484536,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,54226416,(2021 XT1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,28.600,False,False,5.064715,11.325046,2024-Feb-18 11:40,Earth,20.0489643477,72176.2716517384,0.4810100214,187.1128983246,71958074.650094418,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14321,54421913,(2024 AN6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,19.778,False,False,294.411640,658.324441,2024-Feb-12 14:24,Earth,30.8939918704,111218.370733327,0.4812669927,187.2128601603,71996517.009225549,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14322,54423325,(2024 BH5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.841,False,False,45.326791,101.353787,2024-Feb-12 14:56,Earth,11.817004908,42541.2176686402,0.0936693998,36.4373965222,14012742.694258426,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14323,54425949,(2024 CY1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,29.211,False,False,3.822566,8.547517,2024-Feb-12 07:32,Earth,14.5258773843,52293.1585835866,0.0008119411,0.3158450879,121464.659125457,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14324,54426469,(2024 CC4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.493,False,False,8.432473,18.855584,2024-Feb-12 04:17,Earth,7.7637281294,27949.4212658338,0.01176307,4.57583423,1759730.2166609,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [13]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
    'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
    'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
    'miss_distance.astronomical': 'miss_distance_astronomical',
    'miss_distance.lunar': 'miss_distance_lunar',
    'miss_distance.kilometers': 'miss_distance_kilometers',
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
0,3341510,(2006 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.200,False,False,9.650615,21.579430,2024-Feb-18 01:36,Earth,6.5483451916,23574.0426897212,0.1585031059,61.6577081951,23711727.031024433,2024-02-19 17:05:26.730419,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3601342,(2012 EZ11),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.490,False,False,133.830462,299.254010,2024-Feb-18 11:23,Earth,10.3688473103,37327.8503169828,0.2752193896,107.0603425544,41172234.466860152,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3743123,(2016 CA138),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.290,False,False,58.419115,130.629113,2024-Feb-18 16:44,Earth,14.3452175429,51642.7831546067,0.0644683338,25.0781818482,9644325.418929006,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3840791,(2019 GM1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.250,False,False,9.430940,21.088223,2024-Feb-18 22:57,Earth,3.4570904169,12445.5255007355,0.1311391528,51.0131304392,19618137.932484536,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,54226416,(2021 XT1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,28.600,False,False,5.064715,11.325046,2024-Feb-18 11:40,Earth,20.0489643477,72176.2716517384,0.4810100214,187.1128983246,71958074.650094418,2024-02-19 17:05:26.737961,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14321,54421913,(2024 AN6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,19.778,False,False,294.411640,658.324441,2024-Feb-12 14:24,Earth,30.8939918704,111218.370733327,0.4812669927,187.2128601603,71996517.009225549,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14322,54423325,(2024 BH5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.841,False,False,45.326791,101.353787,2024-Feb-12 14:56,Earth,11.817004908,42541.2176686402,0.0936693998,36.4373965222,14012742.694258426,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14323,54425949,(2024 CY1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,29.211,False,False,3.822566,8.547517,2024-Feb-12 07:32,Earth,14.5258773843,52293.1585835866,0.0008119411,0.3158450879,121464.659125457,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...
14324,54426469,(2024 CC4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.493,False,False,8.432473,18.855584,2024-Feb-12 04:17,Earth,7.7637281294,27949.4212658338,0.01176307,4.57583423,1759730.2166609,2024-02-19 17:11:49.935566,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [14]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2023, 2, 19)

In [15]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2024, 2, 19)

In [16]:
delta = (datetime.date.today() - last_execution_date).days
delta

0

In [17]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [18]:
delta = 8

for x in range (0, delta, 7):
    print(x)

0
7


In [19]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links() 
        

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-12&end_date=2024-02-18&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-05&end_date=2024-02-11&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [20]:
delta = 7
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    if delta % 7 == 0:
        # end_date = datetime.date.today()
        # start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        # end_date = end_date.strftime('%Y-%m-%d')
        # link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        # links.append(link)

        for x in range(0, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)  
        
    return links

get_links()

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-11&end_date=2024-02-18&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [21]:
delta = 15

for x in range (0, delta, 7):
    print(x)

def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (datetime.date.today() - datetime.timedelta(days=delta%7-1)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)


    for x in range(delta%7, delta, 7):
        end_date = (datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=x+6)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    return links

get_links() 

0
7
14


['https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-19&end_date=2024-02-19&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-12&end_date=2024-02-18&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-05&end_date=2024-02-11&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [22]:
delta = 14
def get_links(delta):
    if delta == 0:
        return []

    links = []

    def build_link(x, y):
        end_date = x.strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=y)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta % 7 != 0:
        build_link(datetime.date.today(), delta%7-1)
    

    for x in range(delta%7, delta, 7):
        build_link((datetime.date.today() - datetime.timedelta(days=x)), x+6)

    return links

get_links(delta) 

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-13&end_date=2024-02-19&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-06&end_date=2024-02-12&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [23]:
def extract_load(links):
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

            DROP TABLE IF EXISTS neo;                   

            CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                               (datetime.datetime.today(), json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [24]:
def transform():
    df = pd.read_sql_query('''SELECT * FROM neo''', conn)

    df['data'] = df['data'].apply(json.loads)

    json_df = pd.json_normalize(df['data'])

    for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
                                             for k, v in element.items()}

    cad_df = pd.json_normalize(json_df['close_approach_data'])

    df = pd.concat([json_df, cad_df, df], axis=1)

    df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
                  'estimated_diameter.kilometers.estimated_diameter_min',
                  'estimated_diameter.kilometers.estimated_diameter_max',
                  'estimated_diameter.miles.estimated_diameter_min',
                  'estimated_diameter.miles.estimated_diameter_max',
                  'estimated_diameter.feet.estimated_diameter_min',
                  'estimated_diameter.feet.estimated_diameter_max',
                  'close_approach_date', 'epoch_date_close_approach',
                  'relative_velocity.miles_per_hour',
                  'miss_distance.miles'], axis=1
                 )

    df = df.rename(columns={
        'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
        'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
        'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
        'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
        'miss_distance.astronomical': 'miss_distance_astronomical',
        'miss_distance.lunar': 'miss_distance_lunar',
        'miss_distance.kilometers': 'miss_distance_kilometers',
        })

    cols = ['relative_velocity_kilometers_per_second',
            'relative_velocity_kilometers_per_hour',
            'miss_distance_astronomical',
            'miss_distance_lunar',
            'miss_distance_kilometers'
            ]

    df[cols] = df[cols].astype('float')
    
    df['close_approach_date_full'] = df['close_approach_date_full'].astype('datetime64')

    return df

In [25]:
extract_load(get_links(50))
df = transform()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       105 non-null    object        
 1   name                                     105 non-null    object        
 2   nasa_jpl_url                             105 non-null    object        
 3   absolute_magnitude_h                     105 non-null    float64       
 4   is_potentially_hazardous_asteroid        105 non-null    bool          
 5   is_sentry_object                         105 non-null    bool          
 6   estimated_diameter_meters_min            105 non-null    float64       
 7   estimated_diameter_meters_max            105 non-null    float64       
 8   close_approach_date_full                 105 non-null    datetime64[ns]
 9   orbiting_body                            10

C:\Users\Jack\AppData\Local\Temp\ipykernel_19280\311514158.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]


In [26]:
df[df['close_approach_date_full'].duplicated()==True]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
9,2613286,613286 (2005 YQ96),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,20.62,True,False,199.781365,446.724713,2024-01-02 19:42:00,Earth,15.670290,56413.042562,0.167013,64.968073,2.498480e+07,2024-02-19 17:12:09.331739,{'links': {'self': 'http://api.nasa.gov/neo/re...
68,2585310,585310 (2017 YZ1),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,20.43,True,False,218.049440,487.573371,2024-01-06 05:43:00,Earth,12.227294,44018.259382,0.482305,187.616820,7.215187e+07,2024-02-19 17:12:09.332736,{'links': {'self': 'http://api.nasa.gov/neo/re...
69,2591179,591179 (2013 EU9),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.66,False,False,123.752784,276.719637,2024-01-06 16:52:00,Earth,6.456862,23244.704809,0.459550,178.764993,6.874772e+07,2024-02-19 17:12:09.332736,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [27]:
df[df['close_approach_date_full']=='2023-08-20 07:19:00']

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data


In [28]:
df_hazard = df[df['is_potentially_hazardous_asteroid']==1]
df_hazard_wow = df_hazard[df_hazard['miss_distance_lunar']<50]
df_hazard_wow

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data


In [29]:
df[df['estimated_diameter_meters_max']>1000]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
28,2450293,450293 (2004 LV3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.84,True,False,453.476699,1014.004725,2024-01-04 13:59:00,Earth,23.717888,85384.397955,0.251541,97.849296,3.762994e+07,2024-02-19 17:12:09.331739,{'links': {'self': 'http://api.nasa.gov/neo/re...
37,2096590,96590 (1998 XB),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,16.32,False,False,1447.288051,3236.234466,2024-01-03 09:04:00,Earth,10.256351,36922.862281,0.269798,104.951486,4.036123e+07,2024-02-19 17:12:09.331739,{'links': {'self': 'http://api.nasa.gov/neo/re...
78,2138971,138971 (2001 CB21),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.53,True,False,523.064176,1169.607054,2024-01-05 01:38:00,Earth,9.310403,33517.450361,0.407540,158.532944,6.096707e+07,2024-02-19 17:12:09.332736,{'links': {'self': 'http://api.nasa.gov/neo/re...
90,2138893,138893 (2000 YH66),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.16,False,False,620.233153,1386.883492,2024-01-07 11:04:00,Earth,32.999671,118798.816319,0.279440,108.702309,4.180369e+07,2024-02-19 17:12:09.332736,{'links': {'self': 'http://api.nasa.gov/neo/re...
91,2199003,199003 (2005 WJ56),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.17,True,False,617.383440,1380.511341,2024-01-07 02:32:00,Earth,15.290837,55047.012190,0.199535,77.618938,2.984994e+07,2024-02-19 17:12:09.332736,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [30]:
from bokeh.plotting import figure, show

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label="x", y_axis_label="y")

# add a line renderer with legend and line thickness
p.line(x, y, legend_label="Temp.", line_width=2)

# show the results
show(p)

In [31]:
df_diameter = df[['estimated_diameter_meters_min', 'estimated_diameter_meters_max']]
df_diameter

,estimated_diameter_meters_min,estimated_diameter_meters_max
0,350.392641,783.501764
1,16.770846,37.500752
2,199.781365,446.724713
3,101.054342,225.964377
4,16.016034,35.812940
...,...,...
100,133.830462,299.254010
101,48.972805,109.506520
102,7.347760,16.430091
103,21.338062,47.713358


In [32]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

data = df_diameter
source = ColumnDataSource(data)

p = figure()
p.circle(x='estimated_diameter_meters_min', y='estimated_diameter_meters_max', source=source)

show(p)